In [1]:
import numpy
from urllib import urlopen
import scipy.optimize
from math import exp
from math import log
import random
import gzip
from sklearn import svm
from sklearn.decomposition import PCA
from collections import defaultdict
import json
import numpy as np
import time
import sys

In [2]:
# read data and meta
def readGz(f):
    for l in gzip.open(f):
		yield eval(l)

print "Reading data..."
data = json.load(open('data.json'))
meta = json.load(open('meta.json'))
print "Done."

Reading data...
Done.


In [9]:
##################################################
# pre-defined functions                          #
##################################################
def inner(x,y):
    return sum([x[i]*y[i] for i in xrange(len(x))])

def sigmoid(gamma):
    if gamma < 0:
        return 1 - 1/(1 + exp(gamma))
    else:
        return 1/(1 + exp(-gamma))

# NEGATIVE Log-likelihood
def f(theta, X, y, lam):
    loglikelihood = 0
    for i in range(len(X)):
        logit = inner(X[i], theta)
        # To avoid overflow of exp(-logit), for all negative 'logit', replace '-log(1 + exp(-logit))' with 'logit - log(1 + exp(logit))'
        if logit > 0:
            loglikelihood -= log(1 + exp(-logit)) 
        else:
            loglikelihood += logit - log(1 + exp(logit))
        if not y[i]:
            loglikelihood -= logit
    for k in range(len(theta)):
        loglikelihood -= lam * theta[k]*theta[k] 
    return -loglikelihood
 
# NEGATIVE Derivative of log-likelihood
def fprime(theta, X, y, lam):
    dl = [0]*len(theta)
    for i in range(len(X)):
        logit = inner(X[i], theta)
        for k in range(len(theta)):
            dl[k] += X[i][k] * (1 - sigmoid(logit))
            if not y[i]:
                dl[k] -= X[i][k]
    for k in range(len(theta)):
        dl[k] -= lam*2*theta[k]
    return numpy.array([-x for x in dl])



In [3]:
##################################################
#Prepare Training Set and Test Set               #
##################################################
platforms = ['VGA', 'PC', 'Linux', 'Mac', 'Nintendo', 'PlayStation', 'Sony PSP', 'Wii', 'Xbox']
def feature(datum):
	l = meta[datum[1]]
	# Add features that whether each word in platforms appears in categories
	feat = [1] + [any([1 for c in l['categories'][0] if c.startswith(p)]) for p in platforms]
	# Add features price/10 and salesrank
	feat += [l['price']/10.0, l['salesRank'].values()[0]]
	return feat

# All elements in 'data' are reviews from users. Use these data as half training set that responding to y = 1. Fetch all pairs of reviewer and reviewed item from 'data' and store them in 'visited'.
visited = set([])
allReviewers = set([])
for l in data:
	visited.add((l['reviewerID'],l['asin']))
	allReviewers.add(l['reviewerID'])

allReviewers = list(allReviewers)
allItems = meta.keys()
print "visited done"

# To make the training set in balance, generate the data in training set that responding to y = 0. Let the number of generated datas the same with len(data). Store these pairs in 'unvisited'. 
unvisited = set()
while len(unvisited) < len(data):
    reviewer = allReviewers[random.randint(0,len(allReviewers)-1)]
    item = allItems[random.randint(0,len(allItems)-1)]
    if (reviewer, item) not in visited:
        unvisited.add((reviewer, item))
visited = list(visited)
unvisited = list(unvisited)
print "unvisited done"

# Take the first 1/2 of visited and unvisited as the training set and the rest as test set.
trainSet1 = visited[:len(visited)/2]
testSet1 = visited[len(visited)/2:]
trainSet0 = unvisited[:len(unvisited)/2]
testSet0 = unvisited[len(unvisited)/2:]

X_train = [feature(d) for d in trainSet1] + [feature(d) for d in trainSet0]
y_train = [1 for _ in range(len(trainSet1))] + [0 for _ in range(len(trainSet0))]
X_test = [feature(d) for d in testSet1] + [feature(d) for d in testSet0]
y_test = [1 for _ in range(len(testSet1))] + [0 for _ in range(len(testSet0))]
print "Training set and Test set done"

visited done
unvisited done
Training set and Test set done


In [11]:
##################################################
# Train by logistic regression                   #
##################################################
lam = 1.0
print "lambda = " + str(lam)
theta,_,_ = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(X_train[0]), fprime, args = (X_train, y_train, lam))

# check accuracy
predictions = [inner(theta,x) > 0 for x in X_test]
correct = [(a==b) for (a,b) in zip(predictions, y_test)]
acc = sum(correct) * 1.0 / len(correct)
print "Accuracy on testing set by logistic regression = " + str(acc)


lambda = 1.0
Accuracy on testing set by logistic regression = 0.783712028738


In [12]:
##################################################
# Create a support vector classifier object, with regularization parameter C = 1000
##################################################
clf = svm.LinearSVC(C=1000)
clf.fit(X_train, y_train)

# check accuracy
test_predictions = clf.predict(X_test)
correct = [(a==b) for (a,b) in zip(test_predictions, y_test)]
acc = sum(correct) * 1.0 / len(correct)
print "Accuracy on testing set by SVM = " + str(acc)

Accuracy on testing set by SVM = 0.742867854776


In [4]:
class KNN(object):

    def __init__(self, numUsers, numItems, lamI = 6e-2, lamJ = 6e-3, learningRate = 0.1):
        self._numUsers = numUsers
        self._numItems = numItems
        self._lamI = lamI
        self._lamJ = lamJ
        self._learningRate = learningRate
        self._users = set()
        self._items = set()
        self._Iu = defaultdict(set)
        
        
    def sigmoid(self, x):
        return 1/(1+np.exp(-x))

    def train(self, trainData, epochs=30, batchSize=500):
        
        # correlation matrix
        self.C =np.random.rand(self._numItems, self._numItems)  
        for l in xrange(self._numItems):
            self.C[l][l] = 0
            for n in xrange(l, self._numItems):
                self.C[l][n] = self.C[n][l]
              
        # change batch_size to min(batch-size, len(train))
        if len(trainData) < batchSize:
            sys.stderr.write("WARNING: Batch size is greater than number of training samples, switching to a batch size of %s\n" % str(len(trainData)))
            batchSize = len(trainData)
                  
        self._trainDict, self._users, self._items = self._dataPretreatment(trainData)
        N = len(trainData) * epochs
        users, pItems, nItems = self._sampling(N)
        itr = 0
        t2 = t0 = time.time()
        while (itr+1)*batchSize < N:
      
            self._mbgd(
                users[itr*batchSize: (itr+1)*batchSize],
                pItems[itr*batchSize: (itr+1)*batchSize],
                nItems[itr*batchSize: (itr+1)*batchSize]
            )
            
            itr += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.3f%% ) in %.1f seconds" %(str(itr*batchSize), 100.0 * float(itr*batchSize)/N, t2 - t0))
            sys.stderr.flush()
        if N > 0:
            sys.stderr.write("\nTotal training time %.2f seconds; %.2f samples per second\n" % (t2 - t0, N*1.0/(t2 - t0)))
            sys.stderr.flush()
            
            
    def _mbgd(self, users, pItems, nItems):
        
        prev = -2**10
        for _ in xrange(30):
            
            gradientC = defaultdict(float)
            obj = 0

            for ind in xrange(len(users)):
                u, i, j = users[ind], pItems[ind], nItems[ind]
                x_ui = sum([self.C[i][l] for l in self._Iu[u] if i != l])
                x_uj = sum([self.C[j][l] for l in self._Iu[u]])
                x_uij =  x_ui - x_uj
                
                for l in self._Iu[u]:
                    if l != i:
                        gradientC[(i,l)] += (1-self.sigmoid(x_uij)) + self._lamI * self.C[i][l]**2
                        gradientC[(l,i)] += (1-self.sigmoid(x_uij)) + self._lamI * self.C[l][i]**2
                    gradientC[(j,l)] += -(1-self.sigmoid(x_uij)) + self._lamJ * self.C[j][l]**2
                    gradientC[(l,j)] += -(1-self.sigmoid(x_uij)) + self._lamJ * self.C[l][j]**2
                    
                    obj -= 2*self._lamI * self.C[i][l]**2 + 2*self._lamJ * self.C[j][l]**2
                    
                obj += log(self.sigmoid(x_uij))
            
            #print 'OBJ: ', obj
            if prev > obj: 
                break
            prev = obj
            
            for a,b in gradientC:
                self.C[a][b] += self._learningRate * gradientC[(a,b)]
            
        #print _, '\n'
        
    def _sampling(self, N):
        
        sys.stderr.write("Generating %s random training samples\n" % str(N))
        userList = list(self._users)
        userIndex = np.random.randint(0, len(self._users), N)
        pItems, nItems = [], []
        cnt = 0
        for index in userIndex:
            u = userList[index]
            i = self._trainDict[u][np.random.randint(len(self._Iu[u]))]
            pItems.append(i)
            j = np.random.randint(self._numItems)
            while j in self._Iu[u]:
                j = np.random.randint(self._numItems)
            nItems.append(j)
            
            cnt += 1
            if not cnt %10000:
                sys.stderr.write("\rGenerated %s" %(str(cnt)))
                sys.stderr.flush()
        return userIndex, pItems, nItems

    def predictionsKNN(self, K, u):
        #slow
        if K >= self._Iu[u]:
            res = np.sum([self.C[:,l] for l in self._Iu[u]], 0)
        else:
            res = []
            for i in xrange(self._numItems):
                res.append(sum(sorted([self.C[i][l] for l in self._Iu[u]], reverse=True)[:K]))
        return res

    def predictionsAll(self, u):
        
        res = np.sum([self.C[:,l] for l in self._Iu[u]], 0)
        return res

    def prediction(self, u, i):
        
        scores = self.predictions(u)
        return scores[i] > sorted(scores)[self._numItem*0.8]

    def _dataPretreatment(self, data):
        dataDict = defaultdict(list)
        items = set()
        for u, i in data:
            self._Iu[u].add(i)
            dataDict[u].append(i)
            items.add(i)
        return dataDict, set(dataDict.keys()), items

In [5]:
length = len(data)/2
users = set()
items = set()
visited = set()
businessCount = defaultdict(int)

for l in data[:length]:
    user,business = l['reviewerID'],l['asin']
    users.add(user)  
    items.add(business)
    visited.add((user, business))  #visited pair
    businessCount[business] += 1
    
mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count*1.0/length > 0.571: 
        break

users = list(users)
items = list(items)
Iu = defaultdict(set)
Ui = defaultdict(set)
for l in data[:length]:
    Iu[l['reviewerID']].add(l['asin'])
    Ui[l['asin']].add(l['reviewerID'])
    
users = {value:key for key, value in enumerate(users)}
items = {value:key for key, value in enumerate(items)}

In [8]:
len(testSet1)

114692

In [6]:
train = [(users[l['reviewerID']], items[l['asin']]) for l in data[:length]]   
bpr = KNN(len(users), len(items), lamI = 4e-2, lamJ = 4e-3, learningRate = 0.09)
bpr.train(train, epochs=5, batchSize=500)

Generating 573460 random training samples
Processed 573000 ( 99.920% ) in 109.2 seconds
Total training time 109.20 seconds; 5251.31 samples per second


In [7]:
res = []
cnt = 0
for u,i in testSet1 + testSet0:
    cnt += 1
    if u not in users:  #no history of user, use popularity
        if i in return1:
            res.append(1)
        else:
            res.append(0)
        continue
        
    if i not in items:   #no one visited before
        res.append(0)
        continue
        
    scores = bpr.predictionsKNN(10, users[u])
    #scores = bpr.predictionsAll(users[u])
    #thrhd = sorted(scores)[len(items)*7/10]
    score = scores[items[i]]
    
    num = len([1 for x in scores if x > score])

    #if score >= thrhd:
    if num < len(scores)*30/100:
        res.append(1)
    else:
        res.append(0)
        
    sys.stderr.write("\rProcessed %s" %(str(cnt)))
    sys.stderr.flush()

Processed 229376

In [8]:
correct = [(a==b) for (a,b) in zip(res, y_test)]
acc = sum(correct) * 1.0 / len(correct)
print "Accuracy on testing set by knn = " + str(acc)

Accuracy on testing set by knn = 0.731554947163


In [13]:
from theano_bpr import BPR
import theano
import theano.tensor as T
from theano import function, config, shared, sandbox  

bpr = BPR(5, len(users), len(items), lambda_u = 1e-3, lambda_i = 1e-3, lambda_j = 1e-3, lambda_bias = 1)
bpr.train(train, epochs=2, batch_size=500)

WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
ERROR (theano.gpuarray): Could not initialize pygpu, support disabled
Traceback (most recent call last):
  File "/home/sun/anaconda2/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 164, in <module>
    use(config.device)
  File "/home/sun/anaconda2/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 151, in use
    init_dev(device)
  File "/home/sun/anaconda2/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 47, in init_dev
    raise RuntimeError("The new gpu-backend need a c++ compiler.")
RuntimeError: The new gpu-backend need a c++ compiler.
Generating 229384 random training samples
Processed 229000 ( 99.83% ) in 1.4997 seconds
Total training time 695.72 seconds; 3.0

In [14]:
res = []
cnt = 0
for u,i in testSet1 + testSet0:
    cnt += 1
    if u not in users:  #no history of user, use popularity
        if i in return1:
            res.append(1)
        else:
            res.append(0)
        continue
        
    if i not in items:   #no one visited before
        res.append(0)
        continue
        
    scores = bpr.predictions(users[u])
    #scores = bpr.predictionsAll(users[u])
    #thrhd = sorted(scores)[len(items)*7/10]
    score = scores[items[i]]
    
    num = len([1 for x in scores if x > score])

    #if score >= thrhd:
    if num < len(scores)*30/100:
        res.append(1)
    else:
        res.append(0)
        
    sys.stderr.write("\rProcessed %s" %(str(cnt)))
    sys.stderr.flush()
correct = [(a==b) for (a,b) in zip(res, y_test)]
acc = sum(correct) * 1.0 / len(correct)
print "Accuracy on testing set by MF = " + str(acc)

Processed 229376

Accuracy on testing set by MF = 0.792077913019


In [9]:
data[0]

{u'asin': u'B0018YXM3Y',
 u'helpful': [48, 78],
 u'overall': 5.0,
 u'reviewText': u'I have been a huge fan of the Total War series from Creative Assembly, first starting with Shogun nearly a decade ago, then moving onto both Medieval Total Wars, as well as Rome Total War.  However, I have not been nearly as excited about those games as I have been about Empire.  The time period is fascinating, and the warfare spectacular.  I think many a history fan has long imagined himself (or herself) as Major Pitcairn, at the head of a column of redcoats, facing down a ragtag bunch of insolent tea-dumping rebels, or as the Comte de Conflans, at the head of a battle line entering Quiberon Bay amidst a howling gale, exchanging broadsides with the British fleet.  I remember playing MicroProse\'s Napoleonic 2-D RTS game, Fields of Glory, with British troops marching into battle with the cheerful Grenadier March playing, or the French Imperial Guard with Victory at Hand playing mightily.  How the times 